In [16]:
import os
import random
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Dispositivo in uso: {device}")


Dispositivo in uso: cpu


In [17]:
# Specifica il percorso del dataset e delle classi, se necessario per il preprocessing.
# Se devi fare inferenza su nuovi file, imposta dataset_directory con il percorso dei tuoi nuovi file.
# Se non fai più training, puoi utilizzare lo stesso scaler e le stesse classi usate per il training originale.

label_classes = ["destra", "sinistra", "giù", "su", "silenzio"]
target_duration = 1.1  # Durata in secondi
target_sample_rate = 16000

# Percorso al modello pretrained
model_path = "simple_model.pth"


In [18]:
# Definiamo la stessa architettura del modello utilizzata in fase di training,
# in modo da poter caricare correttamente i pesi pre-addestrati.

class SimpleAudioClassifier(nn.Module):
    def __init__(self, num_classes):
        super(SimpleAudioClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Stessa dimensione calcolata in precedenza:
        # Input: 17600 campioni, dopo 3 pool: 17600/2/2/2 = 2200
        # Canali: 64, quindi 64*2200 = 140800 input al primo FC.
        
        self.fc1 = nn.Linear(64 * 2200, 128)
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)

        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = F.relu(self.conv3(x))
        x = self.pool(x)

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Carichiamo il modello
num_classes = len(label_classes)
model = SimpleAudioClassifier(num_classes=num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()
print("Modello caricato correttamente.")


Modello caricato correttamente.


C:\Users\AndreaFumagalli\AppData\Local\Temp\ipykernel_25832\1705195398.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_

In [19]:
# Se disponi di uno scaler salvato in precedenza, caricalo qui.
# Se non hai uno scaler pre-salvato, puoi crearne uno sulla base dei dati di training originali.
# In caso non lo avessi, è possibile utilizzare i dati di training salvati o un fallback (niente scaling).

# Esempio (commentato): caricamento da file se era stato salvato con joblib
# from joblib import load
# scaler = load("scaler.joblib")

# Se non hai uno scaler, lascia scaler = None e non fare scaling.
scaler = None

def preprocess_audio(file_path, target_sr=16000, desired_duration=1.1, scaler=None):
    # Funzione per preprocessare un singolo file audio (simile a quella usata in training)
    try:
        audio_data, original_sr = librosa.load(file_path, sr=None)
        
        # Resample se necessario
        if original_sr != target_sr:
            audio_data = librosa.resample(audio_data, orig_sr=original_sr, target_sr=target_sr)
        
        desired_length = int(desired_duration * target_sr)
        
        # Padding/Troncamento
        if len(audio_data) > desired_length:
            audio_data = audio_data[:desired_length]
        elif len(audio_data) < desired_length:
            audio_data = np.pad(audio_data, (0, desired_length - len(audio_data)), mode="constant")
        
        # Scaling se disponibile
        if scaler is not None:
            audio_data = scaler.transform([audio_data])[0]
        
        return audio_data
    except Exception as e:
        print(f"Errore nel preprocessing del file {file_path}: {e}")
        return None


In [42]:
# In questa cella eseguiamo l'inferenza su uno o più file audio.
# Esempio: passiamo un file audio preprocessato al modello per ottenere la predizione.

def predict_single_file(model, file_path, label_classes, scaler=None, threshold=0.6):
    audio_data = preprocess_audio(file_path, target_sr=target_sample_rate, desired_duration=target_duration, scaler=scaler)
    if audio_data is None:
        return None
    
    # Convertiamo in tensore e modello
    audio_tensor = torch.tensor(audio_data, dtype=torch.float32).unsqueeze(0).unsqueeze(1).to(device)
    with torch.no_grad():
        outputs = model(audio_tensor)
        probs = torch.softmax(outputs, dim=1)
        conf, pred_idx = torch.max(probs, 1)
    
    conf = conf.item()
    predicted_class = label_classes[pred_idx.item()]
    
    if conf >= threshold:
        print(f"File: {file_path} - Predizione: {predicted_class} (Confidenza: {conf:.2f})")
    else:
        print(f"File: {file_path} - Nessuna predizione chiara. Confidenza massima: {conf:.2f}")

# Esempio d'uso (sostituisci con un tuo file audio):
predict_single_file(model, "test_dataset/suu.mp3", label_classes, scaler=scaler)


File: test_dataset/suu.mp3 - Predizione: su (Confidenza: 1.00)


In [31]:
import time
import sounddevice as sd
import soundfile as sf

def record_and_predict(model, label_classes, file_name="temp_recording.wav", sr=16000, record_duration=1.5, wait_seconds=3, threshold=0.6, scaler=None):
    """
    Questa funzione attende wait_seconds secondi, registra l'audio per record_duration secondi,
    lo salva in file_name (.wav), quindi effettua la predizione con il modello fornito.
    """
    model.eval()

    # Timer prima della registrazione
    print(f"Attendi {wait_seconds} secondi prima della registrazione...")
    time.sleep(wait_seconds)
    print("Inizio registrazione...")

    # Registrazione dell'audio
    recorded_audio = sd.rec(int(record_duration * sr), samplerate=sr, channels=1, dtype='float32')
    sd.wait()
    print("Registrazione completata.")

    # Salvare il file in formato wav
    sf.write(file_name, recorded_audio, sr)
    print(f"Audio salvato come {file_name}")

    # Preprocessare l'audio (resample, padding, scaling se necessario)
    # Dal momento che abbiamo già la frequenza di campionamento impostata, non serve resample.
    # Se necessario, scaling:
    audio_data = np.squeeze(recorded_audio)
    
    # Padding/Troncamento a target_duration se differente da record_duration:
    # Supponiamo di voler usare la stessa durata target del training, ad esempio 1.1 secondi.
    target_duration = 1.1
    desired_length = int(target_duration * sr)
    if len(audio_data) > desired_length:
        audio_data = audio_data[:desired_length]
    elif len(audio_data) < desired_length:
        audio_data = np.pad(audio_data, (0, desired_length - len(audio_data)), mode='constant')

    # Applichiamo lo scaling se abbiamo uno scaler
    if scaler is not None:
        audio_data = scaler.transform([audio_data])[0]

    # Creazione del tensore per il modello
    audio_tensor = torch.tensor(audio_data, dtype=torch.float32).unsqueeze(0).unsqueeze(1).to(device)

    # Inferenza
    with torch.no_grad():
        output = model(audio_tensor)
        probabilities = torch.softmax(output, dim=1)
        confidence, predicted_idx = torch.max(probabilities, 1)

    conf_value = confidence.item()
    predicted_label = label_classes[predicted_idx.item()]

    if conf_value >= threshold and predicted_label != "silenzio":
        print(f"Comando riconosciuto: {predicted_label} (Conf: {conf_value:.2f})")
    else:
        print("Nessun comando chiaro rilevato.")

# Esempio di utilizzo:
record_and_predict(model, label_classes, file_name="my_recording.wav", sr=16000, record_duration=1.5, wait_seconds=3, threshold=0.6, scaler=scaler)


Attendi 3 secondi prima della registrazione...
Inizio registrazione...
Registrazione completata.
Audio salvato come rec.wav
Nessun comando chiaro rilevato.


 #### PRETRAINED

In [ ]:
import torch
import librosa
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-italian"

processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name).to("cpu").eval()

commands = ["destra", "sinistra", "su", "giù"]

def transcribe_audio(file_path, processor, model, target_sr=16000):
    audio_data, sr = librosa.load(file_path, sr=None)
    if sr != target_sr:
        audio_data = librosa.resample(audio_data, orig_sr=sr, target_sr=target_sr)
    input_values = processor(audio_data, sampling_rate=target_sr, return_tensors="pt").input_values
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription.lower()

file_to_predict = "rec.wav"
transcription = transcribe_audio(file_to_predict, processor, model)
print(f"Trascrizione ottenuta: '{transcription}'")

detected_command = None
for cmd in commands:
    if cmd in transcription:
        detected_command = cmd
        break

if detected_command is not None:
    print(f"Comando riconosciuto: {detected_command}")
else:
    print("Nessun comando tra destra, sinistra, su, giù è stato rilevato nella trascrizione.")


c:\Users\AndreaFumagalli\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\AndreaFumagalli\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\AndreaFumagalli\.cache\huggingface\hub\models--jonatasgrosman--wav2vec2-large-xlsr-53-italian. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more deta

Trascrizione ottenuta: 'destra'
Comando riconosciuto: destra
